In [1]:
import glob
import os
import pandas as pd
import numpy as np
import pyproj
from tqdm import tqdm
from multiprocessing import Pool
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
df = pd.read_excel('../外部資料集/鄉鎮土地面積及人口密度(97).ods', engine='odf', skiprows=[i for i in range(40)])
#df = df.drop_duplicates(subset=['地址'], keep='first')
#df = df.reset_index()
df

,區 域 別,年底人口數,土地面積,人口密度
0,總 計,23264640,36197.0669,642.72169
1,新 北 市,3995551,2052.5667,1946.612015
2,板橋區,549572,23.1373,23752.641838
3,三重區,379825,16.3170,23277.869706
4,中和區,403109,20.1440,20011.368149
...,...,...,...,...
390,北竿鄉,3181,9.9000,321.313131
391,莒光鄉,1471,4.7000,312.978723
392,東引鄉,1534,3.8000,403.684211
393,東沙群島,…,2.3800,…


In [3]:
infos = []
city1 = ''
for i, row in df.iterrows():
    try:
        name = row['區　域　別'].strip()
        people_cnt = int(row['年底人口數'])
        ground_area = float(row['土地面積'])
        population_density = float(row['人口密度'])
    except:
        print('skiping', name)
        continue
    if name.endswith('市') or name.endswith('縣'):
        if '  ' in name:
            city1 = name.strip().replace(' ', '').replace('臺北市', '台北市').replace('臺中市', '台中市').replace('臺南市', '台南市')
            continue
    if city1 == '':
        continue
    infos.append({
        '縣市': city1,
        '鄉鎮市區': name.replace(' ', '').replace('　', ''),
        '行政區人口數': people_cnt,
        '行政區土地面積': ground_area,
        '行政區人口密度': population_density,
    })
df_population = pd.DataFrame(infos)
df_population

skiping 東沙群島
skiping 南沙群島


,縣市,鄉鎮市區,行政區人口數,行政區土地面積,行政區人口密度
0,新北市,板橋區,549572,23.1373,23752.641838
1,新北市,三重區,379825,16.3170,23277.869706
2,新北市,中和區,403109,20.1440,20011.368149
3,新北市,永和區,212170,5.7138,37132.906297
4,新北市,新莊區,419966,19.7383,21276.705694
...,...,...,...,...,...
365,金門縣,烏坵鄉,670,1.2000,558.333333
366,連江縣,南竿鄉,7797,10.4000,749.711538
367,連江縣,北竿鄉,3181,9.9000,321.313131
368,連江縣,莒光鄉,1471,4.7000,312.978723


In [4]:
df_population['縣市'].unique().tolist()

['新北市',
 '台北市',
 '桃園市',
 '台中市',
 '台南市',
 '高雄市',
 '宜蘭縣',
 '新竹縣',
 '苗栗縣',
 '彰化縣',
 '南投縣',
 '雲林縣',
 '嘉義縣',
 '屏東縣',
 '臺東縣',
 '花蓮縣',
 '澎湖縣',
 '基隆市',
 '新竹市',
 '嘉義市',
 '金門縣',
 '連江縣']

In [5]:
df_population['縣市'].nunique()

22

In [6]:
use_counties = ['台北市', '高雄市', '新北市', '桃園市', '台中市', '台南市', '苗栗縣', '新竹縣', '基隆市',
       '屏東縣', '新竹市', '宜蘭縣', '花蓮縣', '嘉義市', '金門縣', '嘉義縣', '彰化縣', '雲林縣']
df_population = df_population[df_population['縣市'].isin(use_counties)].reset_index(drop=True)

In [7]:
set(use_counties) - set(df_population['縣市'].unique().tolist())

set()

In [8]:
df_population = df_population.drop_duplicates(subset=['縣市', '鄉鎮市區'], keep='first').reset_index(drop=True)
df_population.to_csv('../外部資料集/population_info.csv', index=False, encoding='utf-8')